# Transformer - Attention is all you need

## Máscaras en el Transformer: Padding y Causal Mask

### ¿Por qué se necesitan máscaras?

En un modelo Transformer, las **máscaras** se usan para controlar qué partes de la secuencia pueden "verse" entre sí durante la auto-atención. Esto es fundamental tanto en el **encoder** como en el **decoder**, pero por diferentes razones:

- **Encoder** → necesita una **padding mask** para ignorar los tokens de relleno (`<PAD>`).
- **Decoder** → necesita tanto:
  - una **padding mask**,
  - como una **máscara causal (look-ahead mask)** que impide ver tokens del futuro durante la generación.

---

| Tipo de Máscara         | Código / Ejemplo                                                                                     | Forma                                          | Descripción                                                                                                     |
|-------------------------|------------------------------------------------------------------------------------------------------|------------------------------------------------|-----------------------------------------------------------------------------------------------------------------|
| **Padding Mask (source)**   | `(source != 0).unsqueeze(1).unsqueeze(2)`                                                           | `(batch_size, 1, 1, source_seq_len)`             | Ignora los tokens de padding (0) en el input del encoder.                                                       |
| **Padding Mask (target)**   | `(target != 0).unsqueeze(1).unsqueeze(2)`                                                           | `(batch_size, 1, 1, target_seq_len)`             | Ignora los tokens de padding en el input del decoder.                                                           |
| **Máscara Causal (Look-Ahead)** | `torch.tril(torch.ones(1, size, size)).bool()` <br> *(con `size = target.size(1)`)*                  | `(1, target_seq_len, target_seq_len)`            | Triangular inferior: permite que el token en posición *i* vea solo los tokens hasta la posición *i*.             |
| **Target Mask Combinada**   | `(target != 0).unsqueeze(1).unsqueeze(2) & no_mask` <br> *(donde `no_mask` es la máscara causal)*     | `(batch_size, 1, target_seq_len, target_seq_len)` | Combina la máscara de padding y la causal para el decoder, usando broadcasting para ajustar las dimensiones.     |




In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import Counter
import math
import numpy as np
import re

# Semilla de reproducibilidad
torch.manual_seed(23)

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [9]:
MAX_SEQ_LEN = 30 

In [38]:
class PositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_seq_len = MAX_SEQ_LEN):
        super().__init__()
        self.pos_embed_matrix = torch.zeros(max_seq_len, d_model, device=device) # filas: max_seq_len, columnas: d_model
        token_pos = torch.arange(0, max_seq_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (math.log(10000.0)/d_model))
        
        self.pos_embed_matrix[:, 0::2] = torch.sin(token_pos * div_term)
        self.pos_embed_matrix[:, 1::2] = torch.cos(token_pos * div_term)
        self.pos_embed_matrix = self.pos_embed_matrix.unsqueeze(0).transpose(0,1) # Para hacer broadcasting con x.shape
        
    def forward(self, x):
        # Broadcasting automático
        # x: (seq_len, batch_size, d_model)
        # pos_embed_matrix: (seq_len, d_model)
        # resultado: (seq_len, batch_size, d_model) + (seq_len, 1, d_model) = (seq_len, batch_size, d_model)
        # print(self.pos_embed_matrix.shape)
        # print(x.shape)
        return x + self.pos_embed_matrix[:x.size(0), :]
    
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model=512, num_heads=8): # d_model tiene que ser divisible entre num_heads. d_v = 512/8 = 64. (8*64=512). Siendo 512 el tamaño del embedding y la concatenación de las 8 cabezas igual al tamaño del embedding
        super().__init__()
        assert d_model % num_heads == 0, 'Embedding size not compatible with num heads'
        
        self.d_v = d_model // num_heads
        self.d_k = self.d_v
        self.num_heads = num_heads
        
        self.W_q = nn.Linear(d_model, d_model) # En lugar de hacer 8 de 512x64 hacemos una de 512x512 (más eficiente)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def forward(self, Q, K, V, mask=None):
        batch_size = Q.size(0)
        '''
        Q, K, V -> [batch_size, seq_len, num_heads*d_k]
        Después de view Q: (batch_size, 10, 8, 64)
        Luego de transpose se reorganiza a (batch_size, 8, 10, 64) para aplicar atención
        '''
        Q = self.W_q(Q).view(batch_size, -1, num_heads, self.d_k).transpose(1,2) # Partimos la dimension de 512 en 8 cabezas de 64. Cada token tiene 8 sub-vectores de 64 → 1 por cabeza
        K = self.W_k(K).view(batch_size, -1, num_heads, self.d_k).transpose(1,2)
        V = self.W_v(V).view(batch_size, -1, num_heads, self.d_k).transpose(1,2)
        
        weighted_values, attention = self.scale_dot_product(Q, K, V, mask)
        
        weighted_values = weighted_values.transpose(1,2).contiguous().view(batch_size, -1, self.num_heads*self.d_k) # (batch_size, num_heads, seq_len, d_k)
        weighted_values = self.W_o(weighted_values)
        
        return weighted_values, attention
        
    def scale_dot_product(self, Q, K, V, mask=None):
        scores = torch.matmul(Q, K.transpose(-2,-1)) / math.sqrt(self.d_k)
        if mask is not None: # En el Encoder para el padding, en el Decoder para el padding y para no ver el futuro del output
            scores = scores.masked_fill(mask == 0, -1e9) # Para que al aplicar softmax den probabilidades de 0
            # scores.shape = (batch_size, num_heads, seq_len_q, seq_len_k)
        attention = F.softmax(scores, dim=-1) # dim=-1 normaliza por filas
        weighted_values = torch.matmul(attention, V)
        
        return weighted_values, attention
    
class PositionFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        
    def forward(self, x):
        return self.linear2(F.relu(self.linear1(x)))
    
    
class EncoderSubLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.ffn = PositionFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        
    def forward(self, x, mask=None):
        attn_score, _ = self.self_attn(x, x, x, mask)
        x = x + self.dropout1(attn_score) # Skip connection
        x = self.norm1(x) # Normalización
        x = x + self.dropout2(self.ffn(x)) # Skip connection
        return self.norm2(x) # Normalización 

class Encoder(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, num_layers, dropout=0.1):
        super().__init__()
        self.layers = nn.ModuleList([EncoderSubLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)]) # Nx capas secuenciales
        self.norm = nn.LayerNorm(d_model)
        
    def forward(self, x, mask=None):
        # mask para el padding
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

class DecoderSubLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)
    
    def forward(self, x, encoder_output, target_mask=None, encoder_mask=None):
        attention_score, _ = self.self_attn(x, x, x, target_mask)
        x = x + self.dropout1(attention_score)
        x = self.norm1(x)
        
        encoder_attn, _ = self.cross_attn(x, encoder_output, encoder_output, encoder_mask)
        x = x + self.dropout2(encoder_attn)
        x = self.norm2(x)
        
        ff_output = self.feed_forward(x)
        x = x + self.dropout3(ff_output)
        return self.norm3(x)
        
class Decoder(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, num_layers, dropout=0.1):
        super().__init__()
        self.layers = nn.ModuleList([DecoderSubLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.norm = nn.LayerNorm(d_model)
        
    def forward(self, x, encoder_output, target_mask, encoder_mask):
        # cross-attention
        # Necesitamos el encoder_mask para no atender a los maskings
        for layer in self.layers:
            x = layer(x, encoder_output, target_mask, encoder_mask)
        return self.norm(x)

In [39]:
class Transformer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, num_layers, input_vocab_size, target_vocab_size,
                max_len=MAX_SEQ_LEN, dropout=0.1):
        # d_model: Tamaño de los embeddings
        # num_heads: Número de cabezas paralelas de atención
        # d_ff: Tamaño de las redes neuronales Feed-Forward
        # num_layers: Número de capas secuenciales tanto para el encoder como para el decoder
        # input_vocab_size
        # target_vocab_size
        # max_len: Tamaño de la ventana de contexto
        
        super().__init__()
        self.encoder_embedding = nn.Embedding(input_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(target_vocab_size, d_model)
        self.pos_embedding = PositionalEmbedding(d_model, max_len)
        self.encoder = Encoder(d_model, num_heads, d_ff, num_layers, dropout)
        self.decoder = Decoder(d_model, num_heads, d_ff, num_layers, dropout)
        self.output_layer = nn.Linear(d_model, target_vocab_size)
        
    def forward(self, source, target):
        # Encoder mask
        source_mask, target_mask = self.mask(source, target)
        # Embedding and positional Encoding
        source = self.encoder_embedding(source) * math.sqrt(self.encoder_embedding.embedding_dim) # Técnica de escalado para normalizar los valores de los embeddings
        source = self.pos_embedding(source)
        # Encoder
        encoder_output = self.encoder(source, source_mask)
        
        # Decoder embedding and positional encoding
        target = self.decoder_embedding(target) * math.sqrt(self.decoder_embedding.embedding_dim)
        target = self.pos_embedding(target)
        # Decoder
        output = self.decoder(target, encoder_output, target_mask, source_mask)
        
        return self.output_layer(output)
        
    def mask(self, source, target):
        # El token de 0 es de padding
        # El resto o bien son tokens especiales (<SOS>, <EOS>) o bien palabras (Aqui cada palabra equivale a un token)
        source_mask = (source != 0).unsqueeze(1).unsqueeze(2)
        target_mask = (target != 0).unsqueeze(1).unsqueeze(2)
        size = target.size(1)  # La dimensión 1 representa la longitud de la secuencia (max_seq_len)
        no_mask = torch.tril(torch.ones(1, size, size, device=device)).bool() # Para evitar ver palabras futuras que aún no se han generado
        target_mask = target_mask & no_mask # Broadcasting automático  # (B, 1, T, T)
        return source_mask, target_mask

### Simple test

In [40]:
seq_len_source = 10
seq_len_target = 10
batch_size = 2
input_vocab_size = 50
target_vocab_size = 50

source = torch.randint(1, input_vocab_size, (batch_size, seq_len_source))
target = torch.randint(1, target_vocab_size, (batch_size, seq_len_target))

d_model = 512
num_heads = 8
d_ff = 2048
num_layers = 6

model = Transformer(d_model, num_heads, d_ff, num_layers, input_vocab_size, target_vocab_size,
                max_len=MAX_SEQ_LEN, dropout=0.1)

model = model.to(device)
source = source.to(device)
target = target.to(device)

In [41]:
output = model(source, target)

NameError: name 'output_layer' is not defined